In [13]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import datetime
from pandas import to_datetime
import seaborn as sn
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr, iqr
import pickle

In [14]:
customer_sequence = pd.read_csv("../../preprocessed.csv", index_col=0)
customer_sequence

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,536365,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,71053,WHITE METAL LANTERN,6
2,536365,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,84029E,RED WOOLLY HOTTIE WHITE HEART.,6
...,...,...,...,...,...,...,...,...
541904,581587,2011-09-12 12:50:00,0.85,12680.0,France,22613,PACK OF 20 SPACEBOY NAPKINS,12
541905,581587,2011-09-12 12:50:00,2.10,12680.0,France,22899,CHILDREN'S APRON DOLLY GIRL,6
541906,581587,2011-09-12 12:50:00,4.15,12680.0,France,23254,CHILDRENS CUTLERY DOLLY GIRL,4
541907,581587,2011-09-12 12:50:00,4.15,12680.0,France,23255,CHILDRENS CUTLERY CIRCUS PARADE,4


In [15]:
customer_sequence=customer_sequence.groupby(["CustomerID",'BasketID','BasketDate'])

In [16]:
customer_sequence=customer_sequence["ProdDescr"].apply(list).to_frame()

In [17]:
#customer_sequence=customer_sequence.groupby(["CustomerID"]).apply(list)
customer_sequence=customer_sequence.sort_values(by=['BasketDate'])
customer_sequence

ProdDescr
CustomerID BasketID BasketDate                                                            
17850.0    536365   2010-01-12 08:26:00  [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
           536366   2010-01-12 08:28:00  [HAND WARMER UNION JACK, HAND WARMER RED POLKA...
13047.0    536368   2010-01-12 08:34:00  [JAM MAKING SET WITH JARS, RED COAT RACK PARIS...
           536367   2010-01-12 08:34:00  [POPPY'S PLAYHOUSE BEDROOM , POPPY'S PLAYHOUSE...
           536369   2010-01-12 08:35:00                         [BATH BUILDING BLOCK WORD]
...                                                                                    ...
16945.0    570864   2011-12-10 15:41:00  [FELTCRAFT BUTTERFLY HEARTS, SKULLS  WATER TRA...
           570865   2011-12-10 15:42:00  [PLAYING CARDS KEEP CALM & CARRY ON, PLAYING C...
17096.0    570866   2011-12-10 15:49:00  [RETROSPOT HEART HOT WATER BOTTLE, SCOTTIE DOG...
12667.0    570875   2011-12-10 17:00:00  [BAKING SET SPACEBOY DESIGN, RED RETROSPOT SUG...
16085.0    570876   2011-12-10 17:19:00  [HANGING  BUTTERFLY T-LIGHT HOLDER, CERAMIC HE...

[16376 rows x 1 columns]

In [18]:
customer_sequence=customer_sequence.groupby(["CustomerID"])["ProdDescr"].apply(list)

In [19]:
customer_sequence

CustomerID
12347.0    [[BLACK CANDELABRA T-LIGHT HOLDER, AIRLINE BAG...
12348.0                                          [[POSTAGE]]
12349.0    [[PARISIENNE CURIO CABINET, SWEETHEART WALL TI...
12350.0    [[CHOCOLATE THIS WAY METAL SIGN, METAL SIGN NE...
12352.0    [[DELUXE SEWING KIT , PINK HEART SHAPE EGG FRY...
                                 ...                        
18280.0    [[WOOD BLACK BOARD ANT WHITE FINISH, RETROSPOT...
18281.0    [[ROBOT BIRTHDAY CARD, CARD CIRCUS PARADE, PEN...
18282.0    [[REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEAC...
18283.0    [[COLOUR GLASS T-LIGHT HOLDER HANGING, I CAN O...
18287.0    [[SMALL PURPLE BABUSHKA NOTEBOOK , SMALL RED B...
Name: ProdDescr, Length: 4150, dtype: object

In [20]:
customer_sequence.iloc[0]

[['BLACK CANDELABRA T-LIGHT HOLDER',
  'AIRLINE BAG VINTAGE JET SET BROWN',
  'COLOUR GLASS. STAR T-LIGHT HOLDER',
  'CLEAR DRAWER KNOB ACRYLIC EDWARDIAN',
  'PINK DRAWER KNOB ACRYLIC EDWARDIAN',
  'GREEN DRAWER KNOB ACRYLIC EDWARDIAN',
  'RED DRAWER KNOB ACRYLIC EDWARDIAN',
  'PURPLE DRAWERKNOB ACRYLIC EDWARDIAN',
  'BLUE DRAWER KNOB ACRYLIC EDWARDIAN',
  'ALARM CLOCK BAKELIKE CHOCOLATE',
  'ALARM CLOCK BAKELIKE GREEN',
  'ALARM CLOCK BAKELIKE RED ',
  'ALARM CLOCK BAKELIKE PINK',
  'ALARM CLOCK BAKELIKE ORANGE',
  'FOUR HOOK  WHITE LOVEBIRDS',
  'BATHROOM METAL SIGN ',
  'LARGE HEART MEASURING SPOONS',
  'BOX OF 6 ASSORTED COLOUR TEASPOONS',
  'BLUE 3 PIECE POLKADOT CUTLERY SET',
  'RED 3 PIECE RETROSPOT CUTLERY SET',
  'PINK 3 PIECE POLKADOT CUTLERY SET',
  'EMERGENCY FIRST AID TIN ',
  'SET OF 2 TINS VINTAGE BATHROOM ',
  'SET/3 DECOUPAGE STACKING TINS',
  'BOOM BOX SPEAKER BOYS',
  'RED TOADSTOOL LED NIGHT LIGHT',
  'BLACK EAR MUFF HEADPHONES',
  'CAMOUFLAGE EAR MUFF HEADPHONES'],

In [21]:
customer_sequence=customer_sequence.values.tolist()

In [22]:
customer_sequence

[[['BLACK CANDELABRA T-LIGHT HOLDER',
   'AIRLINE BAG VINTAGE JET SET BROWN',
   'COLOUR GLASS. STAR T-LIGHT HOLDER',
   'CLEAR DRAWER KNOB ACRYLIC EDWARDIAN',
   'PINK DRAWER KNOB ACRYLIC EDWARDIAN',
   'GREEN DRAWER KNOB ACRYLIC EDWARDIAN',
   'RED DRAWER KNOB ACRYLIC EDWARDIAN',
   'PURPLE DRAWERKNOB ACRYLIC EDWARDIAN',
   'BLUE DRAWER KNOB ACRYLIC EDWARDIAN',
   'ALARM CLOCK BAKELIKE CHOCOLATE',
   'ALARM CLOCK BAKELIKE GREEN',
   'ALARM CLOCK BAKELIKE RED ',
   'ALARM CLOCK BAKELIKE PINK',
   'ALARM CLOCK BAKELIKE ORANGE',
   'FOUR HOOK  WHITE LOVEBIRDS',
   'BATHROOM METAL SIGN ',
   'LARGE HEART MEASURING SPOONS',
   'BOX OF 6 ASSORTED COLOUR TEASPOONS',
   'BLUE 3 PIECE POLKADOT CUTLERY SET',
   'RED 3 PIECE RETROSPOT CUTLERY SET',
   'PINK 3 PIECE POLKADOT CUTLERY SET',
   'EMERGENCY FIRST AID TIN ',
   'SET OF 2 TINS VINTAGE BATHROOM ',
   'SET/3 DECOUPAGE STACKING TINS',
   'BOOM BOX SPEAKER BOYS',
   'RED TOADSTOOL LED NIGHT LIGHT',
   'BLACK EAR MUFF HEADPHONES',
   'CAMOU

In [23]:
outfile = open("customer_sequence",'wb')

In [24]:
pickle.dump(customer_sequence,outfile)
outfile.close()

In [11]:
from gsp import *

In [12]:
#signature requires: the dataset, the min support and the boolean verbose
result_set = apriori(customer_sequence, 10, verbose=True)

KeyboardInterrupt: 